下面这段代码，是来自 OpenAI Cookbook 里面的一个例子。它是用同样的方法，来判断亚马逊提供的用户对一些食物的评价，这个评价数据里面，不只有用户给出的评论内容，还有用户给这些食物打了几颗星。这些几颗星的信息，正好可以拿来验证我们这个方法有多准。对于用户打出 1～2 星的，我们认为是差评，对于 4～5 星的，我们认为是好评。我们可以通过 Pandas，将这个 CSV 数据读取到内存里面。为了避免重新调用 OpenAI 的 API 浪费钱，这个数据集里，已经将获取到的 Embedding 信息保存下来了，不需要再重新计算。

In [3]:

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)

# convert 5-star rating to binary sentiment
df = df[df.Score != 3]
df["sentiment"] = df.Score.replace({1: "negative", 2: "negative", 4: "positive", 5: "positive"})

In [4]:
df.head

<bound method NDFrame.head of      Unnamed: 0   ProductId          UserId  Score  \
0             0  B003XPF9BO  A3R7JR3FMEBXQB      5   
1           297  B003VXHGPK  A21VWSCGW7UUAR      4   
2           296  B008JKTTUA  A34XBAIFT02B60      1   
3           295  B000LKTTTW  A14MQ40CCU8B13      5   
4           294  B001D09KAM  A34XBAIFT02B60      1   
..          ...         ...             ...    ...   
995         623  B0000CFXYA  A3GS4GWPIBV0NT      1   
996         624  B0001BH5YM   A1BZ3HMAKK0NC      5   
997         625  B0009ET7TC  A2FSDQY5AI6TNX      5   
998         619  B007PA32L2  A15FF2P7RPKH6G      5   
999         999  B001EQ5GEO  A3VYU0VO6DYV6I      5   

                                               Summary  \
0    where does one  start...and stop... with a tre...   
1                     Good, but not Wolfgang Puck good   
2    Should advertise coconut as an ingredient more...   
3                                     Best tomato soup   
4    Should advertise coconut a

每一条评论都用我们上面的方法，和一个预先设定好的好评和差评的文本去做对比，然后看它离哪个近一些。这里的好评和差评，我们写得稍微长了一点，分别是 “An Amazon review with a negative sentiment.” 和 “An Amazon review with a positive sentiment.”。
在计算完结果之后，我们利用 Scikit-learn 这个机器学习的库，将我们的预测值和实际用户打出的星数做个对比，然后输出对比结果。需要的代码，也就不到 20 行。

In [ ]:

from openai.embeddings_utils import cosine_similarity, get_embedding

from sklearn.metrics import PrecisionRecallDisplay

def evaluate_embeddings_approach(
    labels = ['negative', 'positive'], 
    model = EMBEDDING_MODEL,
):
    label_embeddings = [get_embedding(label, engine=model) for label in labels]

    def label_score(review_embedding, label_embeddings):
        return cosine_similarity(review_embedding, label_embeddings[1]) - cosine_similarity(review_embedding, label_embeddings[0])

    probas = df["embedding"].apply(lambda x: label_score(x, label_embeddings))
    preds = probas.apply(lambda x: 'positive' if x>0 else 'negative')

    report = classification_report(df.sentiment, preds)
    print(report)

    display = PrecisionRecallDisplay.from_predictions(df.sentiment, probas, pos_label='positive')
    _ = display.ax_.set_title("2-class Precision-Recall curve")

evaluate_embeddings_approach(labels=['An Amazon review with a negative sentiment.', 'An Amazon review with a positive sentiment.'])